# Building a Mortgage Pre-approval AI Agent

## Introduction

This notebook demonstrates how to build a production-ready AI agent that automates the mortgage pre-approval process. We'll follow the six-step framework outlined in our comprehensive guide to building secure AI agents:

1. Define scope and requirements
2. Select models and architecture
3. Create tools and integrations
4. Orchestrate workflows
5. Evaluate performance
6. Deploy to production

## Setting Up Cohere API

First, let's set up our Cohere API key. You'll need to generate a key from the [Cohere dashboard](https://dashboard.cohere.com/welcome/register) if you don't already have one.

In [22]:
import os

# Replace this with your Cohere API key
COHERE_API_KEY = "your-api-key-here"

# Alternatively, you can set it as an environment variable
# os.environ["COHERE_API_KEY"] = COHERE_API_KEY

In [23]:
# Install the Cohere Python SDK
!pip install cohere

## Step 1: Define Scope and Setup

In this first step, we define the scope of our mortgage pre-approval agent and set up the data structures it will work with.

The agent will automate these key components of the mortgage pre-approval process:
- Credit check across three bureaus
- Employment verification
- Final report generation

We'll start by importing the necessary libraries and creating a mock customer database. In a production environment, this would connect to your actual customer database.

In [24]:
# Import necessary libraries
import cohere
import json
import datetime

# Initialize Cohere client
co_client = cohere.ClientV2(api_key=COHERE_API_KEY)

# Mock customer database setup
# In production, this would be a real database connection
customer_db = {
    "CUS20240111105523": {
        "intake_date": "2025-03-12",
        "customer_id": "CUS20240111105523",
        # Personal information structure
        "personal_info": {
            "first_name": "Michael",
            "last_name": "Scott",
            "email": "mscott@mail.com",
            "phone": "815-555-5555",
            "address": "123 Drury Lane",
            "city": "Springfield",
            "state": "IL",
            "zip_code": "62701"
        },
        # Document information structure - organized by category
        "document_info": {
            "identity": {
                "ssn": "123-45-6789",
                "dl_number": "M425-7819-3264",
                "dob": "03/15/1965"
            },
            "income": {
                "hourly_rate": 45.67,
                "net_pay": 2643.38,
                "annual_salary": 95000.0,
                "federal_tax_withheld": 19000.0
            },
            "employment": {
                "employer_name": "Dunder Mifflin Paper Company",
                "employer_email": "hr@dundermifflin.com"
            }
        },
        # Empty placeholders for data to be collected during the process
        "credit_info": {},
        "employment_verification": {
            "status": "PENDING"
        }
    }
}

## Step 2: Create Mock Credit Bureau Databases

In a real-world implementation, your agent would connect to APIs from credit bureaus like Experian, Equifax, and TransUnion. For this demonstration, we'll create mock databases that simulate these services.

These mock databases use tuples of (name, SSN) as keys to retrieve credit information, simulating how you might look up a customer in a real credit bureau's system.

In [25]:
# Mock credit bureau databases
# In production, these would be API connections to actual credit bureaus

experian_db = {
    ("Michael Scott", "123-45-6789"): {
        "credit_score": 720,
        "report_date": "2025-03-15"
    }
}

equifax_db = {
    ("Michael Scott", "123-45-6789"): {
        "credit_score": 715,
        "report_date": "2025-03-15"
    }
}

transunion_db = {
    ("Michael Scott", "123-45-6789"): {
        "credit_score": 718,
        "report_date": "2025-03-15"
    }
}

## Step 3: Create Credit Bureau Tools

Now we'll create the functions that will serve as tools for our AI agent. These functions implement what we call a "tool creation pattern" with three components:

1. Tool function (the implementation)
2. Tool definition (to be defined later in `get_tool_definitions`)
3. Function mapping (to be defined later in `run_react_agent`)

These functions abstract away the complexity of connecting to credit bureaus and provide a clean interface for the AI agent to use.

In [26]:
def check_experian(name: str, ssn: str) -> dict:
    """Check credit score with Experian

    Args:
        name (str): Full name of applicant
        ssn (str): Social Security Number

    Returns:
        dict: Credit score information or error message
    """
    try:
        return experian_db[(name, ssn)]
    except KeyError:
        return {"error": "Person not found"}

def check_equifax(name: str, ssn: str) -> dict:
    """Check credit score with Equifax"""
    try:
        return equifax_db[(name, ssn)]
    except KeyError:
        return {"error": "Person not found"}

def check_transunion(name: str, ssn: str) -> dict:
    """Check credit score with TransUnion"""
    try:
        return transunion_db[(name, ssn)]
    except KeyError:
        return {"error": "Person not found"}

## Step 4: Create Customer Record Management Tool

Our agent needs to be able to update the customer database with information it collects during the pre-approval process. The following function provides a way to update a customer's credit information.

This function implements a data persistence pattern with error handling, ensuring that customer data is properly maintained throughout the process.

In [27]:
def update_customer_record(customer_id: str, credit_info: dict) -> dict:
    """Update customer record with credit information

    Implements a data persistence pattern with error handling

    Args:
        customer_id (str): Unique identifier for the customer
        credit_info (dict): Dictionary containing credit scores from bureaus

    Returns:
        dict: Status message indicating success or failure
    """
    if customer_id in customer_db:
        customer_db[customer_id]["credit_info"] = credit_info
        return {"status": "success", "message": f"Updated credit info for {customer_id}"}
    return {"error": "Customer not found"}

## Step 5: Create Employment Verification Tool

A critical part of the pre-approval process is verifying the applicant's employment. The following function creates and sends an employment verification email to the applicant's employer.

This function implements an external communication pattern with templating, making it easy to generate professional emails with the appropriate information.

In [28]:
def send_employment_verification_email(employer_name: str, employee_name: str,
                                   employer_email: str, salary: int) -> dict:
    """Send employment verification email

    Implements an external communication pattern with templating

    Args:
        employer_name (str): Name of the employer company
        employee_name (str): Name of the applicant/employee
        employer_email (str): Email address to send verification request to
        salary (int): Reported annual salary to verify

    Returns:
        dict: Status of the sent email
    """
    email_template = f"""Subject: Employment Verification for {employee_name}
    To: {employer_email}
    From: mortgage.processing@bank.com

    Dear {employer_name} HR Team,

    We are processing a mortgage application for {employee_name} and require employment verification.

    Please verify the following information:
    1. Employment Status
    2. Position/Title
    3. Annual Salary (reported: ${salary:,})
    4. Length of Employment

    Your prompt response is appreciated.

    Best regards,
    Mortgage Processing Team"""

    # Debug logging
    print("\nSending Employment Verification Email:")
    print("=" * 50)
    print(email_template)
    print("=" * 50)

    # Update verification status in database
    customer_db["CUS20240111105523"]["employment_verification"].update({
        "status": "PENDING",
        "request_date": datetime.datetime.now().strftime("%Y-%m-%d"),
        "employer_name": employer_name,
        "reported_salary": salary
    })

    return {"status": "sent", "to": employer_email, "verification_status": "PENDING"}

## Step 6: Create Employment Verification Processing

Once the employer responds to our verification request, we need functions to process that response and update our records. The following functions handle the processing of incoming verification emails and updating the customer database.

These functions implement data processing and state management patterns, ensuring data integrity throughout the process.

In [29]:
def process_verification_email(email_data: dict) -> dict:
    """Process incoming employment verification email

    Implements a data processing pattern for external responses

    Args:
        email_data (dict): Contents of the verification email

    Returns:
        dict: Structured verification data
    """
    return {
        "status": "VERIFIED",
        "verification_date": email_data["date"],
        "employment_status": "ACTIVE",
        "position": "Regional Manager",
        "employer": "Dunder Mifflin Paper Company",
        "salary_verified": 95000,
        "employment_length": "15 years"
    }

def update_employment_verification(customer_id: str, verification_data: dict) -> dict:
    """Update customer record with employment verification results

    Implements state management with data preservation patterns

    Args:
        customer_id (str): Unique identifier for the customer
        verification_data (dict): Employment verification results

    Returns:
        dict: Status message indicating success or failure
    """
    if customer_id in customer_db:
        # Save existing credit info to prevent data loss
        existing_credit_info = customer_db[customer_id].get("credit_info", {})

        # Update employment verification
        customer_db[customer_id]["employment_verification"].update(verification_data)

        # Ensure credit info is preserved
        customer_db[customer_id]["credit_info"] = existing_credit_info

        # Debug logging
        print("\nAfter employment verification update:")
        print("Credit Info:", json.dumps(existing_credit_info, indent=2))
        print("Employment Info:", json.dumps(customer_db[customer_id]["employment_verification"], indent=2))

        return {"status": "success", "message": "Employment verification updated"}
    return {"error": "Customer not found"}

## Step 7: Create Final Report Generation

The final step in our pre-approval process is generating a comprehensive report for the loan officer. This function leverages Cohere's LLM to analyze all the collected data and produce a detailed assessment.

This function implements a report generation pattern using an LLM, allowing for sophisticated analysis and natural language output.

In [30]:
def write_final_report(customer_id: str, co_client) -> dict:
    """Generate comprehensive loan officer report

    Implements a report generation pattern using an LLM

    Args:
        customer_id (str): Unique identifier for the customer
        co_client: Cohere client instance

    Returns:
        dict: Dictionary containing the generated report
    """
    customer = customer_db[customer_id]
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")

    # Create detailed context for the LLM
    system_message = """You are an expert mortgage analyst responsible for creating detailed loan application review reports for loan officers. Your reports help loan officers make informed decisions about mortgage applications. Create a professionally formatted report without using asterisks for emphasis."""

    user_message = f"""Current application details:
    Applicant: {customer['personal_info']['first_name']} {customer['personal_info']['last_name']}
    ID: {customer_id}
    Application Date: {customer['intake_date']}
    Credit Scores: Experian ({customer['credit_info'].get('experian', 0)}), Equifax ({customer['credit_info'].get('equifax', 0)}), TransUnion ({customer['credit_info'].get('transunion', 0)})
    Employment Status: {customer['employment_verification'].get('status', 'PENDING')}
    Employer: {customer['document_info']['employment'].get('employer_name', 'Dunder Mifflin Paper Company')}
    Employment Details: {json.dumps(customer['employment_verification'], indent=2)}
    Income Information: {json.dumps(customer['document_info']['income'], indent=2)}
    Contact Info: {json.dumps(customer['personal_info'], indent=2)}

    IMPORTANT NOTES:
    1. Use today's date ({current_date}) as the report date.
    2. Sign the report as "Mortgage Processing System" rather than using placeholder text.
    3. Format the report professionally without using asterisks for emphasis.

    Write a comprehensive mortgage application review report for a loan officer that:
    1. Analyzes the credit profile and its implications
    2. Evaluates the employment verification status
    3. Reviews income and tax information
    4. Highlights potential concerns or positive factors
    5. Provides context for decision making"""

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    # Generate report using Cohere's Command model
    response = co_client.chat(
        model="command-a-03-2025",
        messages=messages
    )

    return {
        "loan_officer_report": response.message.content[0].text
    }

## Step 8: Define Tool Definitions

For our AI agent to use the tools we've created, we need to define them in a format that the agent can understand. The following function creates these definitions, which include the name, description, and parameter specifications for each tool.

These definitions implement a tool definition pattern that allows the agent to understand the available capabilities and how to use them.

In [31]:
def get_tool_definitions(customer_id):
    """Define available tools for the agent

    Implements a tool definition pattern with clear descriptions
    and parameter specifications

    Args:
        customer_id (str): Unique identifier for the customer

    Returns:
        list: List of tool definitions for the Cohere agent
    """
    return [
        {
            "type": "function",
            "function": {
                "name": "check_experian",
                "description": "Check Experian credit score",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "ssn": {"type": "string"}
                    },
                    "required": ["name", "ssn"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "check_equifax",
                "description": "Check Equifax credit score",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "ssn": {"type": "string"}
                    },
                    "required": ["name", "ssn"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "check_transunion",
                "description": "Check TransUnion credit score",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "ssn": {"type": "string"}
                    },
                    "required": ["name", "ssn"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "update_customer_record",
                "description": "Update customer record with credit information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "customer_id": {"type": "string"},
                        "credit_info": {
                            "type": "object",
                            "properties": {
                                "experian": {"type": "number"},
                                "equifax": {"type": "number"},
                                "transunion": {"type": "number"}
                            }
                        }
                    },
                    "required": ["customer_id", "credit_info"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "send_employment_verification_email",
                "description": "Send employment verification email to the employer's HR department",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "employer_name": {
                            "type": "string",
                            "description": "Name of the company where the applicant works (e.g., Dunder Mifflin)"
                        },
                        "employee_name": {
                            "type": "string",
                            "description": "Full name of the applicant/employee"
                        },
                        "employer_email": {
                            "type": "string",
                            "description": "Email address of the employer's HR department"
                        },
                        "salary": {
                            "type": "number",
                            "description": "Annual salary of the employee to be verified"
                        }
                    },
                    "required": ["employer_name", "employee_name", "employer_email", "salary"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "write_final_report",
                "description": "Generate comprehensive loan officer report",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "customer_id": {"type": "string"}
                    },
                    "required": ["customer_id"]
                }
            }
        }
    ]

## Step 9: Main Agent Implementation

Now we'll implement the main function that orchestrates the entire pre-approval process. This function initializes the Cohere client, sets up the agent with appropriate instructions, and handles the workflow from start to finish.

This function implements the ReAct pattern (Reasoning and Acting) for sequential workflow handling, allowing the agent to make decisions based on the information it gathers along the way.

In [32]:
def run_react_agent(customer_id, max_steps=10, verification_email=None):
    """Main agent orchestration function

    Implements the ReAct pattern for sequential workflow handling

    Args:
        customer_id (str): Unique identifier for the customer
        max_steps (int, optional): Maximum number of steps to execute. Defaults to 10.
        verification_email (dict, optional): Employment verification response email. Defaults to None.

    Returns:
        str: Final response from the agent
    """
    customer = customer_db[customer_id]

    # Define system message with clear instructions
    system_message = f"""You are a mortgage pre-approval agent with access to customer data:
    Customer ID: {customer_id}
    Name: {customer['personal_info']['first_name']} {customer['personal_info']['last_name']}
    SSN: {customer['document_info']['identity']['ssn']}
    Salary: {customer['document_info']['income']['annual_salary']}
    Employer: Dunder Mifflin Paper Company
    Employer Email: {customer['document_info']['employment']['employer_email']}

    Follow these steps:
    1. Check credit scores from all bureaus
    2. Update customer record with credit info
    3. Send employment verification email to the employer's HR department (using the employer_email provided)
    4. Once employment is verified, write final report"""

    # Initialize conversation
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"Process pre-approval for {customer['personal_info']['first_name']} {customer['personal_info']['last_name']}"}
    ]

    # Handle employment verification if provided
    if verification_email:
        print("\n" + "=" * 50)
        print("Processing employer verification response...")
        print("=" * 50 + "\n")

        verification_result = process_verification_email(verification_email)
        update_result = update_employment_verification(customer_id, verification_result)

        print("\n" + "=" * 50)
        print("Employer verification complete. Generating final report...")
        print("=" * 50 + "\n")

        messages.append({
            "role": "user",
            "content": "Employment verification received and processed. Please write the final report."
        })

    # Setup tools and function mapping
    tools = get_tool_definitions(customer_id)
    functions_map = {
        "check_experian": check_experian,
        "check_equifax": check_equifax,
        "check_transunion": check_transunion,
        "update_customer_record": update_customer_record,
        "send_employment_verification_email": send_employment_verification_email,
        "write_final_report": write_final_report
    }

    # Main agent loop
    step = 0
    last_step = None
    while step < max_steps:
        # Get next action from LLM
        response = co_client.chat(
            model="command-a-03-2025",
            messages=messages,
            tools=tools
        )

        # Handle tool calls
        if response.message.tool_calls:
            messages.append({
                "role": "assistant",
                "tool_calls": response.message.tool_calls,
                "tool_plan": response.message.tool_plan
            })

            # Print execution plan at first step
            if step == 0:
                print("\n" + "=" * 50)
                print("EXECUTION PLAN:")
                print("=" * 50)
                print(response.message.tool_plan)
                print("=" * 50 + "\n")

            # Execute tools
            for tool_call in response.message.tool_calls:
                if tool_call.function.name != last_step:
                    print("\n" + "-" * 30)

                params = json.loads(tool_call.function.arguments)

                # Special handling for final report
                if tool_call.function.name == "write_final_report":
                    result = write_final_report(params["customer_id"], co_client)
                else:
                    result = functions_map[tool_call.function.name](**params)
                    print(f"Executing: {tool_call.function.name}")
                    print(f"Parameters: {json.dumps(params, indent=2)}")
                    print(f"Result: {json.dumps(result, indent=2)}")

                last_step = tool_call.function.name

                # Update conversation history
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": [{"type": "document", "document": {"data": json.dumps(result)}}]
                })
        else:
            # Handle completion or waiting state
            if not verification_email:
                print("\n" + "=" * 50)
                print("Waiting for employer verification response...")
                print("=" * 50)
            return response.message.content[0].text

        step += 1

    return "Max steps reached"

## Step 10: Running the Agent

Finally, let's run our agent to see it in action. We'll first run it to initiate the pre-approval process, and then simulate receiving an employment verification response email to complete the process.

In a production environment, you would set up webhooks or scheduled jobs to check for incoming emails and process them automatically.

In [33]:
# Execute the initial processing phase
# The agent will check credit scores and send an employment verification email
print("PHASE 1: INITIAL PROCESSING")
print("=" * 50)
result = run_react_agent("CUS20240111105523")
print(result)

# Simulate receiving an employment verification response
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
incoming_email = {
    "from": "hr@dundermifflin.com",
    "subject": "RE: Employment Verification Request for Michael Scott",
    "date": current_date,
    "body": """Dear Mortgage Processing Team,

    In response to your employment verification request for Michael Scott:
    1. Employment Status: ACTIVE (Full-Time)
    2. Position/Title: Regional Manager
    3. Annual Salary: 95000
    4. Length of Employment: 15 years

    Best regards,
    Toby Flenderson
    HR Department
    Dunder Mifflin Paper Company"""
}

# Execute the final phase with the employment verification
print("\n\nPHASE 2: EMPLOYMENT VERIFICATION AND FINAL REPORT")
print("=" * 50)
result = run_react_agent("CUS20240111105523", verification_email=incoming_email)
print(result)

PHASE 1: INITIAL PROCESSING

EXECUTION PLAN:
I will follow the steps outlined in the preamble to process the pre-approval for Michael Scott.


------------------------------
Executing: check_experian
Parameters: {
  "name": "Michael Scott",
  "ssn": "123-45-6789"
}
Result: {
  "credit_score": 720,
  "report_date": "2025-03-15"
}

------------------------------
Executing: check_equifax
Parameters: {
  "name": "Michael Scott",
  "ssn": "123-45-6789"
}
Result: {
  "credit_score": 715,
  "report_date": "2025-03-15"
}

------------------------------
Executing: check_transunion
Parameters: {
  "name": "Michael Scott",
  "ssn": "123-45-6789"
}
Result: {
  "credit_score": 718,
  "report_date": "2025-03-15"
}

------------------------------
Executing: update_customer_record
Parameters: {
  "customer_id": "CUS20240111105523",
  "credit_info": {
    "experian": 720,
    "equifax": 715,
    "transunion": 718
  }
}
Result: {
  "status": "success",
  "message": "Updated credit info for CUS2024011110